In [6]:
import random, sys, time

###########################################################################
#                                                                         #
# Implement a hash table from scratch! (⑅•ᴗ•⑅)                            #
#                                                                         #
# Please do not use Python's dictionary or Python's collections library.  #
# The goal is to implement the data structure yourself.                   #
#                                                                         #
###########################################################################

# Hash function.
# |key|: string
# Return value: a hash value
def calculate_hash(key):
    assert type(key) == str
    # Note: This is not a good hash function. Do you see why?
    hash = 0
    count = 0
    for i in key:
        count +=1
        hash += ord(i) + hash*count
    return hash


# An item object that represents one key - value pair in the hash table.
class Item:
    # |key|: The key of the item. The key must be a string.
    # |value|: The value of the item.
    # |next|: The next item in the linked list. If this is the last item in the
    #         linked list, |next| is None.
    def __init__(self, key, value, next):
        assert type(key) == str
        self.key = key
        self.value = value
        self.next = next


# The main data structure of the hash table that stores key - value pairs.
# The key must be a string. The value can be any type.
#
# |self.bucket_size|: The bucket size.
# |self.buckets|: An array of the buckets. self.buckets[hash % self.bucket_size]
#                 stores a linked list of items whose hash value is |hash|.
# |self.item_count|: The total number of items in the hash table.
class HashTable:

    # Initialize the hash table.
    def __init__(self):
        # Set the initial bucket size to 97. A prime number is chosen to reduce
        # hash conflicts.
        self.bucket_size = 97
        self.buckets = [None] * self.bucket_size
        self.item_count = 0

    # Put an item to the hash table. If the key already exists, the
    # corresponding value is updated to a new value.
    #
    # |key|: The key of the item.
    # |value|: The value of the item.
    # Return value: True if a new item is added. False if the key already exists
    #               and the value is updated.
    def reHashtable(self,new_size):
        old_buckets = self.buckets
        self.buckets = [None]*new_size
        self.bucket_size = new_size
        self.item_count = 0
        for item in old_buckets:
            if item is not None:
                self.put(item.key,item.value)


    def put(self, key, value):
        assert type(key) == str

        self.check_size() # Note: Don't remove this code.      
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]
        new_item = Item(key,value,None)
        if item is None:
            self.buckets[bucket_index] = new_item
            self.item_count += 1
        else:
            while item:
                if item.key == key:
                    item.value = value
                    return False
                if item.next == None:
                    item.next = new_item
                    self.item_count += 1
                item = item.next
                
        if self.item_count > self.bucket_size*0.7:
            self.reHashtable(self.bucket_size*2)

 
        return True

    # Get an item from the hash table.
    #
    # |key|: The key.
    # Return value: If the item is found, (the value of the item, True) is
    #               returned. Otherwise, (None, False) is returned.
    def get(self, key):
        assert type(key) == str
        self.check_size() # Note: Don't remove this code.
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]
        while item:
            if item.key == key:
                return (item.value, True)
            item = item.next
        return (None, False)

    # Delete an item from the hash table.
    #
    # |key|: The key.
    # Return value: True if the item is found and deleted successfully. False
    #               otherwise.
    def delete(self, key):
        assert type(key) == str
        #------------------------#
        # Write your code here!  #
        self.check_size()
        bucket_index = calculate_hash(key) % self.bucket_size
        item = self.buckets[bucket_index]
        if item:
            if item.key == key:
               self.buckets[bucket_index] = item.next
               self.item_count -=1
            return True
        while item:
            if item.next.key == key:
                item.next = item.next.next
                self.item_count -= 1
                if self.item_count < self.bucket_size*0.3 and self.item_count>97:
                    self.reHashtable((self.bucket_size+1)//2)
                return True
            item = item.next

        return False
                
        #------------------------#

    # Return the total number of items in the hash table.
    def size(self):
        return self.item_count

    # Check that the hash table has a "reasonable" bucket size.
    # The bucket size is judged "reasonable" if it is smaller than 100 or
    # the buckets are 30% or more used.
    #
    # Note: Don't change this function.
    def check_size(self):
        assert (self.item_count < self.bucket_size*0.7
        or self.item_count >= self.bucket_size * 0.3)



# Test the functional behavior of the hash table.
def functional_test():
    hash_table = HashTable()

    assert hash_table.put("aaa", 1) == True
    assert hash_table.get("aaa") == (1, True)
    assert hash_table.size() == 1

    assert hash_table.put("bbb", 2) == True
    assert hash_table.put("ccc", 3) == True
    assert hash_table.put("ddd", 4) == True
    assert hash_table.get("aaa") == (1, True)
    assert hash_table.get("bbb") == (2, True)
    assert hash_table.get("ccc") == (3, True)
    assert hash_table.get("ddd") == (4, True)
    assert hash_table.get("a") == (None, False)
    assert hash_table.get("aa") == (None, False)
    assert hash_table.get("aaaa") == (None, False)
    assert hash_table.size() == 4

    assert hash_table.put("aaa", 11) == False
    assert hash_table.get("aaa") == (11, True)
    assert hash_table.size() == 4

    assert hash_table.delete("aaa") == True
    assert hash_table.get("aaa") == (None, False)
    assert hash_table.size() == 3

    assert hash_table.delete("a") == False
    assert hash_table.delete("aa") == False
    assert hash_table.delete("aaa") == False
    assert hash_table.delete("aaaa") == False

    assert hash_table.delete("ddd") == True
    assert hash_table.delete("ccc") == True
    assert hash_table.delete("bbb") == True
    assert hash_table.get("aaa") == (None, False)
    assert hash_table.get("bbb") == (None, False)
    assert hash_table.get("ccc") == (None, False)
    assert hash_table.get("ddd") == (None, False)
    assert hash_table.size() == 0

    assert hash_table.put("abc", 1) == True
    assert hash_table.put("acb", 2) == True
    assert hash_table.put("bac", 3) == True
    assert hash_table.put("bca", 4) == True
    assert hash_table.put("cab", 5) == True
    assert hash_table.put("cba", 6) == True
    assert hash_table.get("abc") == (1, True)
    assert hash_table.get("acb") == (2, True)
    assert hash_table.get("bac") == (3, True)
    assert hash_table.get("bca") == (4, True)
    assert hash_table.get("cab") == (5, True)
    assert hash_table.get("cba") == (6, True)
    assert hash_table.size() == 6

    assert hash_table.delete("abc") == True
    assert hash_table.delete("cba") == True
    assert hash_table.delete("bac") == True
    assert hash_table.delete("bca") == True
    assert hash_table.delete("acb") == True 
    assert hash_table.delete("cab") == True
    assert hash_table.size() == 0

    print("Functional tests passed!")


# Test the performance of the hash table.
#
# Your goal is to make the hash table work with mostly O(1).
# If the hash table works with mostly O(1), the execution time of each iteration
# should not depend on the number of items in the hash table. To achieve the
# goal, you will need to 1) implement rehashing (Hint: expand / shrink the hash
# table when the number of items in the hash table hits some threshold) and
# 2) tweak the hash function (Hint: think about ways to reduce hash conflicts).
def performance_test():
    hash_table = HashTable()

    for iteration in range(100):
        begin = time.time()
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.put(str(rand), str(rand))
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.get(str(rand))
        end = time.time()
        print("%d %.6f" % (iteration, end - begin))

    for iteration in range(100):
        random.seed(iteration)
        for i in range(10000):
            rand = random.randint(0, 100000000)
            hash_table.delete(str(rand))
    print(hash_table.size())
    assert hash_table.size() == 0

    print("Performance tests passed!")


if __name__ == "__main__":
    functional_test()
    performance_test()


Functional tests passed!
0 0.324542
1 0.266524
2 0.394633
3 0.208357
4 0.495451
5 0.212888
6 0.211186
7 0.226917
8 0.653802
9 0.259085
10 0.222054
11 0.212167
12 0.211304
13 0.208929
14 0.277684
15 0.211838
16 0.228004
17 1.275265
18 0.216107
19 0.210259
20 0.202858
21 0.209563
22 0.216473
23 0.209071
24 0.212617
25 0.301163
26 0.225079
27 0.224705
28 0.221308
29 0.226357
30 0.333263
31 0.205875
32 0.207398
33 0.208761
34 2.220079
35 0.230458
36 0.206972
37 0.212747
38 0.201211
39 0.229950
40 0.204335
41 0.205095
42 0.206043
43 0.211905
44 0.213383
45 0.352496
46 0.211433
47 0.202979
48 0.196874
49 0.202835
50 0.212495
51 0.208351
52 0.213618
53 0.395751
54 0.211953
55 0.206251
56 0.214373
57 0.210464
58 0.214421
59 0.204207
60 0.219860
61 0.214361
62 0.208995
63 0.499408
64 0.216345
65 0.204643
66 0.215840
67 0.212191
68 0.225840
69 4.071607
70 0.199365
71 0.195952
72 0.220833
73 0.212021
74 0.196725
75 0.199457
76 0.201755
77 0.204312
78 0.197286
79 0.239407
80 0.209081
81 0.204862
8